In [1]:
import torch.nn as nn
import torch
from torch.nn.parameter import Parameter

/nix/store/v1bkq59yxcf4ys3np121vpmb2abgnh72-python3.12-torch-2.7.0/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /build/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [57]:
a=torch.tensor([[1,2],[3,4]])
b=torch.tensor([[1,2]])

In [58]:
a + b

tensor([[2, 4],
        [4, 6]])

In [61]:
a.float().pow(2).mean(dim=-1, keepdim=True)

tensor([[ 2.5000],
        [12.5000]])

In [62]:
a * torch.rsqrt(b + 1e-6)

tensor([[1.0000, 1.4142],
        [3.0000, 2.8284]])

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], bias=False)
        self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], bias=False)

    def forward(self, x):
        x_fc1 = self.fc1(x)
        x_fc2 = self.fc2(x)
        x = nn.functional.gelu(x_fc1, approximate="tanh") * x_fc2
        return self.fc3(x)

In [84]:
# Set seed for reproducible results
torch.manual_seed(1)
import random
random.seed(1)

In [90]:
# Create test data
input_data = torch.tensor([[1.0, 2.0], [3.0, 4.0]])  # 2x2 input
print(f"Input: {input_data}")
print(f"Input shape: {input_data.shape}")

# Create Linear layer: 2 inputs -> 3 outputs, no bias
linear = nn.Linear(2, 3, bias=False, dtype=torch.float32)

# Print the randomly initialized weights
print(f"\nWeight matrix shape: {linear.weight.shape}")
print(f"Weight matrix:\n{linear.weight}")

known_weights = torch.tensor([
  [  0.8250,   0.9742],
  [  0.7971,   0.6912],
  [  0.1964,   0.4280]
])

# Override the randomly initialized weights
linear.weight.data = known_weights


# Forward pass
output = linear(input_data)
print(f"\nOutput shape: {output.shape}")
print(f"Output:\n{output}")

# Manual computation to verify
manual_output = input_data @ linear.weight.T
print(f"\nManual computation (input @ weight.T):\n{manual_output}")
print(f"Results match: {torch.allclose(output, manual_output)}")

Input: tensor([[1., 2.],
        [3., 4.]])
Input shape: torch.Size([2, 2])

Weight matrix shape: torch.Size([3, 2])
Weight matrix:
Parameter containing:
tensor([[ 0.1975,  0.6707],
        [ 0.4667, -0.6443],
        [-0.6723, -0.3411]], requires_grad=True)

Output shape: torch.Size([2, 3])
Output:
tensor([[2.7734, 2.1795, 1.0524],
        [6.3718, 5.1561, 2.3012]], grad_fn=<MmBackward0>)

Manual computation (input @ weight.T):
tensor([[2.7734, 2.1795, 1.0524],
        [6.3718, 5.1561, 2.3012]], grad_fn=<MmBackward0>)
Results match: True


In [93]:

# Create test data
input_data = torch.tensor([[1.0, 2.0], [3.0, 4.0]])  # 2x2 input
print(f"Input: {input_data}")
print(f"Input shape: {input_data.shape}")

# Create Linear layer: 2 inputs -> 3 outputs, no bias
linear = nn.Linear(2, 3, bias=True, dtype=torch.float32)

known_weights = torch.tensor([
  [  0.8250,   0.9742],
  [  0.7971,   0.6912],
  [  0.1964,   0.4280]
])

# Override the randomly initialized weights
linear.weight.data = known_weights
linear.bias.data = torch.tensor([
  [  0.8250,   0.9742,   0.7971]
])


# Forward pass
output = linear(input_data)
print(f"\nOutput shape: {output.shape}")
print(f"Output:\n{output}")


Input: tensor([[1., 2.],
        [3., 4.]])
Input shape: torch.Size([2, 2])

Output shape: torch.Size([2, 3])
Output:
tensor([[3.5984, 3.1537, 1.8495],
        [7.1968, 6.1303, 3.0983]], grad_fn=<AddmmBackward0>)


In [68]:
out_features = 3
in_features = 2

weight = Parameter(torch.empty((out_features, in_features)))
bias = Parameter(torch.empty(out_features))

In [64]:
weight

Parameter containing:
tensor([[1.5378e-13, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.5661e-41]], requires_grad=True)

In [69]:
bias

Parameter containing:
tensor([1.1390e-13, 0.0000e+00, 0.0000e+00], requires_grad=True)